# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,693.032149,-0.426025,1234,692.606124,693.458173,1385.212248,1386.916346
6,847.687665,-0.446036,1234,847.241628,848.133701,1694.483257,1696.267401
7,245.801637,0.622481,1234,246.424118,245.179156,492.848237,490.358312
8,540.589387,-0.979575,1234,539.609812,541.568963,1079.219624,1083.137925
9,566.439303,0.669812,1234,567.109115,565.769491,1134.218229,1131.538982
10,868.116334,-0.259293,1234,867.857041,868.375626,1735.714082,1736.751253
11,802.549560,-0.997515,1234,801.552045,803.547075,1603.104090,1607.094149
12,491.929451,-0.715319,1234,491.214132,492.644771,982.428264,985.289541
13,305.804674,0.898940,1234,306.703614,304.905734,613.407227,609.811468
14,970.190875,-0.779782,1234,969.411092,970.970657,1938.822185,1941.941313


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-692.606124,693.458173
6,-847.241628,848.133701
7,-246.424118,245.179156
8,-539.609812,541.568963
9,-567.109115,565.769491
10,-867.857041,868.375626
11,-801.552045,803.547075
12,-491.214132,492.644771
13,-306.703614,304.905734
14,-969.411092,970.970657


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-692.606124,693.458173
6,-847.241628,848.133701
7,-246.424118,245.179156
8,-539.609812,541.568963
9,-567.109115,565.769491
10,-867.857041,868.375626
11,-801.552045,803.547075
12,-491.214132,492.644771
13,-306.703614,304.905734
14,-969.411092,970.970657


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-692.60612404,  693.45817311],
       [-847.24162847,  848.13370057],
       [-246.42411847,  245.1791558 ],
       [-539.60981205,  541.56896257],
       [-567.10911467,  565.76949096],
       [-867.8570409 ,  868.37562648],
       [-801.55204519,  803.54707456],
       [-491.21413202,  492.64477062],
       [-306.70361358,  304.90573375],
       [-969.4110924 ,  970.97065671]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,1383.212248,-1386.916346
6,1692.483257,-1696.267401
7,490.848237,-490.358312
8,1077.219624,-1083.137925
9,1132.218229,-1131.538982
10,1733.714082,-1736.751253
11,1601.104090,-1607.094149
12,980.428264,-985.289541
13,611.407227,-609.811468
14,1936.822185,-1941.941313


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.693032,0.286988
6,0.847688,0.276982
7,0.245802,0.811241
8,0.540589,0.010212
9,0.566439,0.834906
10,0.868116,0.370354
11,0.802550,0.001243
12,0.491929,0.142340
13,0.305805,0.949470
14,0.970191,0.110109


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,693.032149,-0.426025
6,847.687665,-0.446036
7,245.801637,0.622481
8,540.589387,-0.979575
9,566.439303,0.669812
10,868.116334,-0.259293
11,802.549560,-0.997515
12,491.929451,-0.715319
13,305.804674,0.898940
14,970.190875,-0.779782


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
